# Module 2 - Build RAG-powered Q&A Application with **RetrieveAndGenerate API**

----

This notebook provides sample code and step-by-step instructions for building a fully-managed question-answering (Q&A) application using a **RetrieveAndGenerate API** of Amazon Bedrock Knowledge Bases.

----

### Introduction

In the previous notebook, we demonstrated how to create a Knowledge Base in Amazon Bedrock — including setting up an S3 data source, configuring an Amazon OpenSearch Serverless (AOSS) vector index, and ingesting documents for retrieval-augmented generation (RAG).

In this notebook, we take the next step: building a Q&A application that can query that Knowledge Base using the `RetrieveAndGenerate` API. This API allows you to retrieve the most relevant content from your Knowledge Base based on a user’s query and automatically pass that information to a foundation model (FM) to generate a grounded, context-aware response.

This is a classic example of the RAG pattern — where external data is dynamically retrieved at query time and incorporated into the model’s prompt to improve relevance, accuracy, and transparency. In this solution, retrieved knowledge base content comes with source attribution, helping end users understand the origin of the response and minimizing the risk of model hallucinations.

![BKB illustration](./images/retrieve_and_generate_api.png)

### Pre-requisites

In order to run this notebook, you should have successfully completed the previous notebook lab:
- [1_create-kb-and-ingest-documents.ipynb](./1\_create-kb-and-ingest-documents.ipynb).

Also, please make sure that you have enabled the following model access in _Amazon Bedrock Console_:
- `Amazon Nova Micro`
- `Amazon Titan Text Embeddings V2`


## 1. Setup

### 1.1 Import the required libraries

In [1]:
# Standard library imports
import os
import sys
import json
import time

# Third-party imports
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError

# Local imports
import utility

# Print SDK versions
print(f"Python version: {sys.version.split()[0]}")
print(f"Boto3 SDK version: {boto3.__version__}")

Python version: 3.12.9
Boto3 SDK version: 1.38.26


### 1.2 Initial setup for clients and global variables

In [7]:
bedrock_kb_name="knowledge-base-quick-start-xpwy7"

boto_session = boto3.Session()
aws_region = boto_session.region_name

def get_knowledge_base_id(kb_name, region="us-east-1"):
    """获取指定名称的知识库 ID"""
    bedrock = boto3.client('bedrock-agent', region_name=aws_region)
    
    # 获取知识库列表
    response = bedrock.list_knowledge_bases()
    
    # 查找匹配名称的知识库
    for kb in response['knowledgeBaseSummaries']:
        if kb['name'] == kb_name:
            return kb['knowledgeBaseId']
    
    # 处理分页结果
    while 'nextToken' in response:
        response = bedrock.list_knowledge_bases(nextToken=response['nextToken'])
        for kb in response['knowledgeBaseSummaries']:
            if kb['name'] == kb_name:
                return kb['knowledgeBaseId']
    
    return None

bedrock_kb_id = get_knowledge_base_id(bedrock_kb_name)
print(f"Knowledge Base ID: {bedrock_kb_id}")

# bedrock_kb_id = aws bedrock list-knowledge-bases --region us-east-1 --profile global_ruiliang | jq -r '.knowledgeBaseSummaries[] | select(.name=="YOUR_KB_NAME") | .knowledgeBaseId'

Knowledge Base ID: VVVSF4ZHVN


In [8]:
# %store -r bedrock_kb_id

In [9]:
# Create boto3 session and set AWS region
boto_session = boto3.Session()
aws_region = boto_session.region_name

# Create boto3 clients for Bedrock
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_agent_client = boto3.client('bedrock-agent-runtime', config=bedrock_config)

# Set the Bedrock model to use for text generation
model_id = 'amazon.nova-micro-v1:0'
model_arn = f'arn:aws:bedrock:{aws_region}::foundation-model/{model_id}'

# Print configurations
print("AWS Region:", aws_region)
print("Bedrock Knowledge Base ID:", bedrock_kb_id)

AWS Region: us-east-1
Bedrock Knowledge Base ID: VVVSF4ZHVN


## 2. Fully-managed RAG with **RetreiveAndGenerate API**

The `RetrieveAndGenerate` API provides a fully managed way to implement the Retrieval-Augmented Generation (RAG) pattern with Amazon Bedrock Knowledge Bases.

When a user submits a query, the API automatically converts the query into vector embeddings, performs a similarity search against the Knowledge Base, and retrieves the most relevant document chunks. These search results are then injected into the foundation model's prompt as additional context, enabling the model to generate more accurate and grounded responses.

For multi-turn conversations, Knowledge Bases also maintain short-term conversational memory — allowing the API to return more contextually relevant answers across a dialogue.

The output of the `RetrieveAndGenerate` API includes:

- The **generated response** from the foundation model

- **Source attribution** metadata for the retrieved content

- The **actual retrieved text chunks** from the Knowledge Base

This makes it easy to build RAG-powered applications with trusted, explainable answers — without having to manage retrieval pipelines or prompt construction yourself.

### 2.1 Retrieve and Generate Example

Let’s now see the `RetrieveAndGenerate` API in action and showcase a fully managed RAG workflow in Amazon Bedrock.

In this example, we’ll use the Knowledge Base built in the previous lab — containing Amazon Shareholder Letters — to demonstrate how the API retrieves relevant information and generates a grounded response to a user query.

In [10]:
user_query = "How does Amazon use technology to better serve its customers?"

response = bedrock_agent_client.retrieve_and_generate(
    input={
        'text': user_query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': bedrock_kb_id,
            'modelArn': model_arn
        }
    }
)

print("Final reply:\n", response['output']['text'])

Final reply:
 Amazon uses technology to better serve its customers through various innovative initiatives and investments. Here are some key examples:

1. **Fulfillment Network:** Amazon has been constantly innovating in its fulfillment network for nearly two decades to shorten the time to get items to customers. This includes building out an expansive set of fulfillment centers, a substantial logistics and transportation capability, and reconfiguring operations in its facilities.

2. **AWS (Amazon Web Services):** AWS is developed by considering the services customers might want, and it involves iterative innovation to improve long-term customer experiences, customer loyalty, and returns for shareholders.

3. **Advertising:** Amazon uses machine learning algorithms to tailor sponsored products to be relevant to customer searches, leading to more useful advertising for customers and better performance for brands. Amazon Marketing Cloud (AMC) is a secure digital environment where advert

### 2.2 Understanding Citations

Citations play a critical role in retrieval-augmented generation (RAG) systems by helping users verify the accuracy of a response and providing transparency into the source of information. Let's now look at the `citations` past of the Knowledge Base response:

In [11]:
print("Citations:\n", json.dumps(response["citations"], indent=2, default=str))

Citations:
 [
  {
    "generatedResponsePart": {
      "textResponsePart": {
        "span": {
          "end": 470,
          "start": 0
        },
        "text": "Amazon uses technology to better serve its customers through various innovative initiatives and investments. Here are some key examples:\n\n1. **Fulfillment Network:** Amazon has been constantly innovating in its fulfillment network for nearly two decades to shorten the time to get items to customers. This includes building out an expansive set of fulfillment centers, a substantial logistics and transportation capability, and reconfiguring operations in its facilities"
      }
    },
    "retrievedReferences": [
      {
        "content": {
          "text": "through the pandemic the same way without the dedication and extraordinary efforts shown by our teams during this period, and I'm eternally grateful. It's not normal for a company of any size to be able to respond to something as discontinuous and unpredictable as thi

Here, the response includes a `generatedResponsePart` field, which contains the natural language answer generated by the model. Each `generatedResponsePart` is paired with `retrievedReferences`, which lists the specific pieces of content from the knowledge base that were used to ground that part of the response. These references include the original source text (`content.text`), as well as metadata like the source URI and page number, so users can easily trace information back to its original document. This structure ensures that answers are both helpful and verifiable, allowing users to explore the source material directly and build trust in the response.

## 3. Conclusions and Next Steps

In this notebook, we built a fully-managed RAG-powered Q&A application using the `RetrieveAndGenerate` API from Amazon Bedrock Knowledge Bases.

We demonstrated how this API simplifies the RAG workflow by automatically retrieving relevant content from a knowledge base and generating grounded, context-aware responses using a foundation model. The responses also include source references, allowing users to easily trace answers back to the original documents.

This approach enables you to quickly build reliable, transparent Q&A solutions without managing the complexity of prompt engineering or retrieval logic manually.

### Next Steps

If you are looking for more flexibility and control over your RAG workflow, Amazon Bedrock Knowledge Bases also provides a `Retrieve` API. This API allows you to perform semantic and/or keyword search over your knowledge base and retrieve the most relevant document chunks, which you can then use to build custom prompts or workflows tailored to your application needs.

To explore this approach, check out the next notebook:

&nbsp; **NEXT ▶** [3_customized-rag-with-retrieve-api](./3\_customized-rag-with-retrieve-api.ipynb)